# ON TEST DAY:

1. Use only batteries over 4.2
2. Charge the batteries for about 15 minutes before each test
3. Upon adding battery to the drone do not connect battery to wire until the battery is fully pushed down
4. Get on the ground and connect battery to the drone
5. When the drone does the little propeller test, it needs to be on the ground or else it won't work
6. Ensure that the red light is blinking twice a second and then run
7. Upon each run, hit cap release, restart, and hit run all

# Import necessary functions

In [79]:
# Code adapted from: https://github.com/bitcraze/crazyflie-lib-python/blob/master/examples/autonomousSequence.py

import time
import numpy as np
import cv2
import matplotlib.pyplot as plt

# CrazyFlie imports:

import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.crazyflie.syncLogger import SyncLogger
from cflib.positioning.position_hl_commander import PositionHlCommander

# Group # and camera

In [80]:
group_number = 1 #1

# Possibly try 0, 1, 2 ...
camera = 0

# Basic functions

In [81]:
# Global flag for net tracking
switch_dir_go_left = False
target_height = 0.8

# Get the current crazyflie position:
def position_estimate(scf):
    log_config = LogConfig(name='Kalman Variance', period_in_ms=500)
    log_config.add_variable('kalman.varPX', 'float')
    log_config.add_variable('kalman.varPY', 'float')
    log_config.add_variable('kalman.varPZ', 'float')

    with SyncLogger(scf, log_config) as logger:
        for log_entry in logger:
            data = log_entry[1]
            x = data['kalman.varPX']
            y = data['kalman.varPY']
            z = data['kalman.varPZ']
            
    print(x, y, z)
    return x, y, z

def revamped_position_estimate(scf):
    log_config = LogConfig(name='Kalman Variance', period_in_ms=500)
    log_config.add_variable('kalman.varPX', 'float')
    log_config.add_variable('kalman.varPY', 'float')
    log_config.add_variable('kalman.varPZ', 'float')

    with SyncLogger(scf, log_config) as logger:
        log_entry = next(logger)  # Get the first log entry
        data = log_entry[1]
        x = data['kalman.varPX']
        y = data['kalman.varPY']
        z = data['kalman.varPZ']
    
    print(f"Position Estimate - X: {x}, Y: {y}, Z: {z}")
    return x,y,z    


# Set the built-in PID controller:
def set_PID_controller(cf):
    # Set the PID Controller:
    print('Initializing PID Controller')
    cf.param.set_value('stabilizer.controller', '1')
    cf.param.set_value('kalman.resetEstimation', '1')
    time.sleep(0.1)
    cf.param.set_value('kalman.resetEstimation', '0')
    time.sleep(2)
    return


#Ascend and hover
def ascend_and_hover(cf):
    # Ascend:
    for y in range(5):
        cf.commander.send_hover_setpoint(0, 0, 0, y / 20) #originally 10 in denominator
        time.sleep(0.1)
    # Hover at 0.5 meters:
    for _ in range(20):
        cf.commander.send_hover_setpoint(0, 0, 0, target_height)
        time.sleep(0.1)
    return

# Hover, descend, and stop all motion:
def hover_and_descend(cf):
    print('Descending:')
    # Hover at 0.5 meters:
    for _ in range(30):
        cf.commander.send_hover_setpoint(0, 0, 0, target_height)
        time.sleep(0.1)
    # Descend:
    for y in range(10):
        cf.commander.send_hover_setpoint(0, 0, 0, (10 - y) / 25) #originally 25 in denominator
        time.sleep(0.1)
    # Stop all motion:
    for i in range(10):
        cf.commander.send_stop_setpoint()
        time.sleep(0.1)
    return


# Sort through contours in the image
def findGreatestContour(contours):
    largest_area = 0
    largest_contour_index = -1
    i = 0
    total_contours = len(contours)

    while i < total_contours:
        area = cv2.contourArea(contours[i])
        if area > largest_area:
            largest_area = area
            largest_contour_index = i
        i += 1
    return largest_area, largest_contour_index


# Startup Check

In [82]:
import math

def get_red_total(frame):
    # These define the upper and lower HSV for the red obstacles.
    lb1 = (145, 35, 75)
    ub1 = (180, 255, 255)
    lb2 = (0, 75, 75)
    ub2 = (20, 255, 255)

    # Convert the frame from BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Compute mask of red obstacles in either color range
    mask1 = cv2.inRange(hsv, lb1, ub1)
    mask2 = cv2.inRange(hsv, lb2, ub2)

    # Combine the masks
    mask = cv2.bitwise_or(mask1, mask2)
    
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # get sum of all bounding boxes
    sum_area = 0
    i = 0
    total_contours = len(contours)
    
    while i < total_contours:
        sum_area += cv2.contourArea(contours[i])
        i += 1
    
    print(f"Total area is {sum_area}")
    return sum_area
    
def start_sequence_check(cf, cap):
    global switch_dir_go_left
    #########
    # Center
    #########
    for _ in range(15):
        cf.commander.send_hover_setpoint(0, 0, 0, target_height)
        time.sleep(0.1)
    _, frame = cap.read()
    center_total = get_red_total(frame)
    _, frame = cap.read()
    center_total_2 = get_red_total(frame)
    center_total_max = max(center_total, center_total_2)
    print(f"Center check completed and yielded {center_total_max}")
    
    #########
    # Right
    #########
    print("Sweeping right")
    for _ in range(20):  # Send the position setpoint repeatedly for 2 seconds
        cf.commander.send_position_setpoint(0, -0.5, target_height, 0)
        time.sleep(0.1)
    #time.sleep(0.1)
    print(f"Should have swept right")
    for _ in range(15):
        cf.commander.send_hover_setpoint(0, 0, 0, target_height)
        time.sleep(0.1)
    _, frame = cap.read()
    right_total = get_red_total(frame)
    _, frame = cap.read()
    right_total_2 = get_red_total(frame)
    right_total_max = max(right_total, right_total_2)
    print(f"Right check completed and yielded {right_total_max}")

    #########
    # Left
    #########
    print("Sweeping left")
    for _ in range(20):  # Send the position setpoint repeatedly for 2 seconds
        cf.commander.send_position_setpoint(0, 0.5, target_height, 0)
        time.sleep(0.1)
    #time.sleep(0.1)
    print(f"Should have swept left")
    for _ in range(15):
        cf.commander.send_hover_setpoint(0, 0, 0, target_height)
        time.sleep(0.1)
    _, frame = cap.read()
    left_total = get_red_total(frame)
    _, frame = cap.read()
    left_total_2 = get_red_total(frame)
    left_total_max = max(left_total, left_total_2)
    print(f"Left check completed and yielded {left_total_max}")
    print("################################################################")

    #########
    # Min
    #########
    min_total = min(center_total_max, right_total_max, left_total_max)

    #########
    # Adjust according to min
    #########
    if min_total == center_total_max:
        print("Center position is best:", center_total_max)
        current_y = 0
        for _ in range(20):  # Send the position setpoint repeatedly for 2 seconds
            cf.commander.send_position_setpoint(0, current_y, target_height, 0)
            time.sleep(0.1)
        time.sleep(0.1)
        print("Now we should have stayed at center")

    elif min_total == right_total_max:
        print("Right position is best:", right_total_max)
        current_y = -0.5
        for _ in range(20):  # Send the position setpoint repeatedly for 2 seconds
            cf.commander.send_position_setpoint(0, current_y, target_height, 0)
            time.sleep(0.1)
        time.sleep(0.1)
        print("Now we should have moved right")

    else:
        print("Left position is best:", left_total_max)
        current_y = 0.5
        print("Now we should have moved left")
        # cf.commander.send_position_setpoint(-0.25, 0, target_height, 0)
        switch_dir_go_left = True
        
    return current_y

# Obstacle Detection

In [83]:
def check_contours(frame):
    # These define the upper and lower HSV for the red obstacles.
    lb1 = (145, 35, 75)
    ub1 = (180, 255, 255)
    lb2 = (0, 75, 75)
    ub2 = (20, 255, 255)

    # Convert the frame from BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Compute mask of red obstacles in either color range
    mask1 = cv2.inRange(hsv, lb1, ub1)
    mask2 = cv2.inRange(hsv, lb2, ub2)
    # Combine the masks
    mask = cv2.bitwise_or(mask1, mask2)
    
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the largest contour
    largest_area, largest_contour_index = findGreatestContour(contours)
    # if largest_area > 23000:
    #     return 3
    # elif largest_area > 10000 and largest_area <= 23000: #22000 older thresshold ... #25000: even Older threshhold
    #     return 2
    # elif largest_area > 5000 and largest_area <= 10000:
    #     return 1
    # else:
    #     return 0
    print(f"Current largest area is {largest_area}")
    if largest_area > 23000: #23000
        return 3
    elif largest_area > 10000 and largest_area <= 23000: #22000 older thresshold ... #25000: even Older threshhold
        return 2
    elif largest_area > 5000 and largest_area <= 10000:
        return 1
    else:
        return 0
    


# Book Detection

In [84]:
def checkforbook(frame, cf, cap):
    lb1, ub1 = (100, 150, 50), (130, 255, 255)
    lb2, ub2 = (90, 100, 50), (140, 255, 255)
    
    def createMask(frame, lb1, ub1, lb2, ub2):
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        mask1 = cv2.inRange(hsv, lb1, ub1)
        mask2 = cv2.inRange(hsv, lb2, ub2)
        return cv2.bitwise_or(mask1, mask2)

    def captureFrame(cap):
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture frame.")
            return None
        return frame
    
    #############################
    # Start first run through
    #############################
    if frame is None:
        return None, None, None, None
    
    mask = createMask(frame, lb1, ub1, lb2, ub2)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # print("the initial frame is", frame)
    largest_area, largest_contour_index = findGreatestContour(contours)
    print(f"startin: largest area is {largest_area} and index is {largest_contour_index}")

    # #############################
    # # Sweep Left if largest_contour_index is -1
    # #############################
    # # Find contour
    # #if largest_contour_index != -1 and largest_area > 1:
    # if largest_contour_index == -1:
    #     print("Rotating left")
    #     duration = 1.0 

    #     # Maintain the turn for the specified duration
    #     start_time = time.time()
    #     while time.time() - start_time < duration:
    #         cf.commander.send_hover_setpoint(0, 0, -45, 0.825)
    #         time.sleep(0.1)
        
    #     # Stop motion after turn and hover for 2 seconds
    #     for _ in range(20):
    #         cf.commander.send_hover_setpoint(0, 0, 0, 0.825)
    #         time.sleep(0.1)

    #     # Capture a new frame from the camera (assuming your camera capture is available)
    #     #ret, frame = cap.read()
    #     frame = captureFrame(cap)
    #     mask = createMask(frame, lb1, ub1, lb2, ub2)
    #     contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #     # print("attemmpting to recompute frame for left rotation", frame)
    #     largest_area, largest_contour_index = findGreatestContour(contours)

    # #############################
    # # Sweep Right if largest_contour_index is still -1
    # #############################
    # if largest_contour_index == -1:
    #     print("Rotating right")
    #     # Maintain the turn for the specified duration
    #     start_time = time.time()
    #     while time.time() - start_time < duration:
    #         cf.commander.send_hover_setpoint(0, 0, 90, 0.825)
    #         time.sleep(0.1)
    #     # Stop motion after turn and hover for 2 seconds
    #     for _ in range(20):
    #         cf.commander.send_hover_setpoint(0, 0, 0, 0.825)
    #         time.sleep(0.1)
            
    #     frame = captureFrame(cap)
    #     mask = createMask(frame, lb1, ub1, lb2, ub2)
    #     contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #     print("attemmpting to recompute frame for right rotation", frame)
    #     largest_area, largest_contour_index = findGreatestContour(contours)    
    
    #############################
    # If no detection
    #############################
    if largest_contour_index == -1:
        print("book probs not on table (book not detected)")

    #############################
    # Compute largest contour
    #############################
    global center_x, center_y, w, h
    # If there's a largest contour, draw the bounding box around it
    #print("Test5: The largest contour index is", largest_contour_index)
    if largest_contour_index != -1:
        #print(f"The largest_area is {largest_area}")
        largest_contour = contours[largest_contour_index]
        x, y, w, h = cv2.boundingRect(largest_contour)
        center_x = x + w // 2
        center_y = y + h // 2
    
        # Draw the bounding box for the largest contour
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    return center_x, center_y, w, h


# Obstacle Avoidance Custom Code [Need to test edge cases]

In [85]:
# Follow the setpoint sequence trajectory:
def left_step(cf, current_x, current_y):
    print('Adjusting left to avoid obstacle')
    steps_per_meter = int(15) #originally 5

    for i in range(1): #Range indicates how many steps are taken with this speed
        current_y = current_y + 1.0/float(steps_per_meter)
        position = [current_x, current_y, target_height, 0.0]
        print('Setting position {}'.format(position))
        for i in range(10):
            cf.commander.send_position_setpoint(position[0],
                                                position[1],
                                                position[2],
                                                position[3])
            time.sleep(0.1)

    # Make sure that the last packet leaves before the link is closed.
    time.sleep(0.1)
    return current_y

# Follow the setpoint sequence trajectory:
def right_step(cf, current_x, current_y):
    print('Adjusting right to avoid obstacle')
    steps_per_meter = int(15) #originally 5

    for i in range(1): #Range indicates how many steps are taken with this speed
        current_y = current_y - 1.0/float(steps_per_meter)
        position = [current_x, current_y, target_height, 0.0]
        #print('Setting position {}'.format(position))
        for i in range(10):
            cf.commander.send_position_setpoint(position[0],
                                                position[1],
                                                position[2],
                                                position[3])
            time.sleep(0.1)
    # Make sure that the last packet leaves before the link is closed.
    time.sleep(0.1)
    return current_y

def forward_step(cf, current_x, current_y, x_direc_speed):
    steps_per_meter = int(x_direc_speed)
    for i in range(1): 
        current_x = current_x + 1.0/float(steps_per_meter)
        position = [current_x, current_y, target_height, 0.0]
        #print('Setting position {}'.format(position))
        for i in range(10):
            cf.commander.send_position_setpoint(position[0],
                                                position[1],
                                                position[2],
                                                position[3])
            time.sleep(0.1)
    # Make sure that the last packet leaves before the link is closed.
    time.sleep(0.1)
    return current_x

def backward_step(cf, current_x, current_y):
    steps_per_meter = int(4)

    for i in range(1): 
        current_x = current_x - 1.0/float(steps_per_meter)
        position = [current_x, current_y, target_height, 0.0]
        #print('Setting position {}'.format(position))
        for i in range(10):
            cf.commander.send_position_setpoint(position[0],
                                                position[1],
                                                position[2],
                                                position[3])
            time.sleep(0.1)
    # Make sure that the last packet leaves before the link is closed.
    time.sleep(0.1)
    return current_x

# def turn_left_ninety(cf):
#     print("Turn left 90 degrees")
#     yaw_rate = -90  # Negative for left turn (degrees per second)
#     duration = 1.0 

#     # Maintain the turn for the specified duration
#     start_time = time.time()
#     while time.time() - start_time < duration:
#         cf.commander.send_hover_setpoint(0, 0, yaw_rate, target_height)
#         time.sleep(0.1)
    
#     # Stop motion after turn
#     cf.commander.send_hover_setpoint(0, 0, 0, target_height)

#     for _ in range(10):
#         cf.commander.send_hover_setpoint(0, 0, 0, target_height)  # Send hover command at 0.5 meters.
#         time.sleep(0.1)  # Waits 0.1 seconds before the next command.

#     print("Turn completed")

# def turn_right_ninety(cf):
#     print("Turn right 90 degrees")
#     yaw_rate = 90  # positive for right turn (degrees per second)
#     duration = 1.0 

#     # Maintain the turn for the specified duration
#     start_time = time.time()
#     while time.time() - start_time < duration:
#         cf.commander.send_hover_setpoint(0, 0, yaw_rate, target_height)
#         time.sleep(0.1)
    
#     # Stop motion after turn
#     cf.commander.send_hover_setpoint(0, 0, 0, target_height)

#     for _ in range(10):
#         cf.commander.send_hover_setpoint(0, 0, 0, target_height)  # Send hover command at 0.5 meters.
#         time.sleep(0.1)  # Waits 0.1 seconds before the next command.
#     print("Turn completed")

def right_boundary_turn_checker(cf, current_x, current_y, frame):
    global switch_dir_go_left
    # turn_left_ninety(cf)
    # if check_contours(frame) == 3: #2 or check_contours(frame) == 3:
    #     turn_right_ninety(cf)
    #     current_x = backward_step(cf, current_x, current_y)
    #     print("Moving backward now")
    # else:
    #     switch_dir_go_left = True  # Set flag when close to the net, and now the drone should attempt to keep moving left
    #     turn_right_ninety(cf)
    #     current_y = left_step(cf, current_x, current_y)
    switch_dir_go_left = True  # Set flag when close to the net, and now the drone should attempt to keep moving left
    current_y = left_step(cf, current_x, current_y)
    return current_x, current_y, switch_dir_go_left

def left_boundary_turn_checker(cf, current_x, current_y, frame):
    global switch_dir_go_left
    # turn_right_ninety(cf)
    # if check_contours(frame) == 3: #2 or check_contours(frame) == 3:
    #     turn_left_ninety(cf)
    #     current_x = backward_step(cf, current_x, current_y)
    #     print("Moving backward now")
    # else:
    #     switch_dir_go_left = False
    #     turn_left_ninety(cf)
    #     current_y = right_step(cf, current_x, current_y)
    switch_dir_go_left = False  # Set flag when close to the net, and now the drone should attempt to keep moving right
    current_y = right_step(cf, current_x, current_y)
    return current_x, current_y, switch_dir_go_left


# Follow the setpoint sequence trajectory:
def movestep(frame, cf, current_y, current_x):
    global switch_dir_go_left
    contour_var = check_contours(frame)
    print(f"Current position is: x (forward/back): {current_x}, y (left/right): {current_y}")
    if (contour_var == 3):
        print("AIGHT WE GOTTA MOVE OUTTA THE WAY")
        # adjusted net size to (-1.05, 1.05) since it was changing direction too early
        if current_y > -1.1 and current_y < 1.1:  # Object detected and within reasonable range
            if switch_dir_go_left:  # Non-default behavior, move left
                print("Flag is up, Move left to avoid net")
                current_y = left_step(cf, current_x, current_y)
                # for _ in range(2):
                #     cf.commander.send_hover_setpoint(0, 0, 0, target_height) #(vx, vy, yawrate, zdistance)
                #     time.sleep(0.1)

            else:  # Default behavior: move right
                print("Flag is down, Move right to avoid obstacle")
                current_y = right_step(cf, current_x, current_y)
                # for _ in range(2):
                #     cf.commander.send_hover_setpoint(0, 0, 0, target_height) #(vx, vy, yawrate, zdistance)
                #     time.sleep(0.1)
                

        elif current_y <= -1.1:  # Too far right
            print("Move left (critical adjustment, with FLAG UP NOW)")
            current_x, current_y, switch_dir_go_left = right_boundary_turn_checker(cf, current_x, current_y, frame)
            # for _ in range(2):
            #     cf.commander.send_hover_setpoint(0, 0, 0, target_height) #(vx, vy, yawrate, zdistance)
            #     time.sleep(0.1)

        elif current_y >= 1.1:  # Too far left
            print("Move right (critical adjustment, with FLAG DOWN NOW)")
            current_x, current_y, switch_dir_go_left = left_boundary_turn_checker(cf, current_x, current_y, frame)
            # for _ in range(2):
            #     cf.commander.send_hover_setpoint(0, 0, 0, target_height) #(vx, vy, yawrate, zdistance)
            #     time.sleep(0.1)

    else:    
        # lower = faster
        if (contour_var == 0):
            x_direc_speed = 4.8 #4.7
        elif (contour_var == 1):
            x_direc_speed = 4.8 #4.7
        elif (contour_var == 2):
            x_direc_speed = 4.8 #4.7
        else:
            x_direc_speed = 0

        current_x = forward_step(cf, current_x, current_y, x_direc_speed) #Otherwise then you gotta move forward
    return current_x, current_y



# Book Movement Code

In [86]:
def horizontal_controller(cf, box_x, x_cur, y_cur):
    """
    cf: crazyflie instance
    box_x: x coordinate of the center of the bounding box in the image
    box_y: y coordinate of the center of the bounding box in the image
    box_width: width of the bounding box in the image
    box_height: height of the bounding box in the image
    x_cur: current x position
    y_cur: current y position
    
    Return True to indicate that the drone is close to the target and thus exit the loop to stop and descend, new x, new y
    Return False to indicate continuing to follow the target, new x, new y.
    """
    image_width = 640
    
    # The center of the image is the width/2
    x_center_of_image = image_width//2
    
    # We know the box_x is the center of the bounding box
    error_for_horizontal_movement = box_x - x_center_of_image
    x_command = x_cur
    print("horizontal error is:", error_for_horizontal_movement)
    if error_for_horizontal_movement >= 250:
        y_command = y_cur -(0.0002 * error_for_horizontal_movement)
        print("Correction 1: move right: Y command: ", y_command)
    elif error_for_horizontal_movement >= 100 and error_for_horizontal_movement < 250:
        y_command = y_cur -(0.0004 * error_for_horizontal_movement)
        print("Correction 1: move right: Y command: ", y_command)
    elif error_for_horizontal_movement >= 50 and error_for_horizontal_movement < 100:
        y_command = y_cur -(0.0006 * error_for_horizontal_movement)  #0.6 i the k_p to move position
        print("Correction 1: move right: Y command: ", y_command)
    elif error_for_horizontal_movement > 10 and error_for_horizontal_movement < 50:
        y_command = y_cur -(0.001 * error_for_horizontal_movement)  #0.6 i the k_p to move position
        print("Correction 1: move right: Y command: ", y_command)
    elif error_for_horizontal_movement > -50 and error_for_horizontal_movement < -10:
    #elif error_for_horizontal_movement < -15:
        y_command = y_cur -(0.001 * error_for_horizontal_movement)
        print("Correction 2: move left: Y command: ", y_command)
    elif error_for_horizontal_movement > -100 and error_for_horizontal_movement <= -50:
        y_command = y_cur -(0.0006 * error_for_horizontal_movement)
        print("Correction 2: move left: Y command: ", y_command)
    elif error_for_horizontal_movement > -250 and error_for_horizontal_movement <= -100:
        y_command = y_cur -(0.0004 * error_for_horizontal_movement)
        print("Correction 2: move left: Y command: ", y_command)
    elif error_for_horizontal_movement <= -250:
        y_command = y_cur -(0.0002 * error_for_horizontal_movement)
        print("Correction 2: move left: Y command: ", y_command)
    elif error_for_horizontal_movement >= -15 and error_for_horizontal_movement <= 15: # correct y position
        print("y coordinate muy bien")
        y_command = y_cur
        print("DROP DROP DROP A FIFTY BAG FOR THE MOB IN THE SPOT") #Elite Drake reference to tell us that the drone is going to stop drop and no roll
        # cf.commander.send_position_setpoint(x_cur, y_cur, target_height, 0)  # Stop command (stay in position)
        cf.commander.send_hover_setpoint(0, 0, 0, 0)
        return x_command, y_command, True
    else:
        print("error is insane we def did something wrong")

    # Send the new position setpoint to Crazyflie
    cf.commander.send_position_setpoint(x_command, y_command, target_height, 0)  # Set position to move towards
    time.sleep(0.5)

    # Return False to continue the loop, with the new command positions
    return x_command, y_command, False

In [87]:
def forward_controller(cf, box_width, box_height, x_cur, y_cur):
    """
    cf: crazyflie instance
    box_x: x coordinate of the center of the bounding box in the image
    box_y: y coordinate of the center of the bounding box in the image
    box_width: width of the bounding box in the image
    box_height: height of the bounding box in the image
    x_cur: current x position
    y_cur: current y position
    
    Return True to indicate that the drone is close to the target and thus exit the loop to stop and descend, new x, new y
    Return False to indicate continuing to follow the target, new x, new y.
    """

    image_width = 640
    image_height = 480

    # If im w 640px, center @ 320px
    # If im h 480px, center @ 240px
    current_box_size = box_width * box_height  # Area of the bounding box
    total_image_size = image_width * image_height # Area of the image

    y_command = y_cur

    #We use the error ratio of the image sizes for depth perception
    error_for_forwards_and_backwards_movement = current_box_size/total_image_size
    print("forward error:", error_for_forwards_and_backwards_movement)
    
    # If the bounding box to total image ratio is greater than the threshold, stop the drone
    #print(error_for_forwards_and_backwards_movement)
    if error_for_forwards_and_backwards_movement > 0.04: # make this lower to land earlier (book is smaller in frame) 
        print("DROP DROP DROP A FIFTY BAG FOR THE MOB IN THE SPOT") #Elite Drake reference to tell us that the drone is going to stop drop and no roll
        # cf.commander.send_position_setpoint(x_cur, y_cur, target_height, 0)  # Stop command (stay in position)
        cf.commander.send_hover_setpoint(0, 0, 0, 0)
        return x_cur, y_cur, True # If the target is big enough, we are close, so stop the drone and return True
    else: # correct x position
        x_command = x_cur + (10 * error_for_forwards_and_backwards_movement) #10000 is the k_p to move position
        print("new x:", x_command)

    # Calculate the desired position adjustments based on the error
    # Adjust in x direction (this is forward and backwards on drone facing the human)
    # As reflected in video, we specifically chose to keep depth perception and x-direction movement small compared to left 
    # to right motion to avoid collision with a human or with net.
    print("Suspected MOVE FORWARD")
    
    # Send the new position setpoint to Crazyflie
    cf.commander.send_position_setpoint(x_command, y_command, target_height, 0)  # Set position to move towards
    time.sleep(0.5)

    # Return False to continue the loop, with the new command positions
    return x_command, y_command, False

# Runner

In [88]:
# Set the URI the Crazyflie will connect to
uri = f'radio://0/{group_number}/2M'

# Initialize all the CrazyFlie drivers:
cflib.crtp.init_drivers(enable_debug_driver=False)

# Scan for Crazyflies in range of the antenna:
print('Scanning interfaces for Crazyflies...')
available = cflib.crtp.scan_interfaces()

# List local CrazyFlie devices:
print('Crazyflies found:')
for i in available:
    print(i[0])

# Check that CrazyFlie devices are available:
if len(available) == 0:
    print('No Crazyflies found, cannot run example')
else:
    ## Ascent to hover; run the sequence; then descend from hover:
    # Use the CrazyFlie corresponding to team number:
    with SyncCrazyflie(uri, cf=Crazyflie(rw_cache='./cache')) as scf:
        # Get the Crazyflie class instance:
        cf = scf.cf
        current_x = 0.0
        current_y = 0.0

        # Initialize and ascend:
        t = time.time()
        elapsed = time.time() - t
        ascended_bool = 0

        cap = cv2.VideoCapture(camera)
        # reached_height = False
        horizontal_achieved = False
        # forward_achieved = False
        centered = False
        first_frame_for_book = False
        start_sequence_done = False
        while(cap.isOpened()):

            ret, frame = cap.read() # Read a frame from the video capture.
            elapsed = time.time() - t
            
            if(elapsed > 5.0):

                #print('Capturing.....')
                if ret:  # If a frame was successfully read.
                    if(ascended_bool == 0):         # If the Crazyflie has not ascended yet.
                        set_PID_controller(cf)      # Set up the PID controller for stabilization.
                        ascend_and_hover(cf)        # Ascend and hover.
                        print("PID is done")
                        ascended_bool = 1           # Update the flag to indicate ascent is done.
                    else:
                        # perform timestep movement computation
                        if not start_sequence_done:
                            current_y = start_sequence_check(cf, cap) # change starting position so we tackle less obstacles
                            start_sequence_done = True
                            ret, frame = cap.read() 
                            
                        if current_x < 3.54: #3.4: #3.35:
                            ret, frame = cap.read() # Read a frame from the video capture.
                            current_x, current_y = movestep(frame, cf, current_y, current_x)

                        else: # past all obstacles
                            if not centered:
                                print("Beyond boundary now .. lets first center!")
                                # if not reached_height:
                                    #############
                                    #Hover to 1.2 m - working as of 12/5 - 7:50 pm. will have to edit current_y when we do movestep
                                    #############
                                    # target_height = 0.8                     # Desired hover height (0.825 FOR USUAL SPOT)
                                    # current_height = 0.5                    # Starting hover height is 0.5 due to PID
                                    # step_delay = 1                          # Delay (in seconds) between each step
                                    # h_moves = np.linspace(current_height, target_height, num = 3) # Creating a step array for it to traverse through to hover slowly
                                    # for height in h_moves:                                        # For each step, move
                                    #     #print("moving to height: ", height)                         
                                    #     cf.commander.send_hover_setpoint(0, 0, 0, height)         #This is the def send_hover_setpoint(self, vx, vy, yawrate, zdistance) - https://www.bitcraze.io/documentation/repository/crazyflie-lib-python/master/api/cflib/crazyflie/commander/
                                    #     time.sleep(step_delay) #Makes it sleep for a quick second
                                
                                # cf.commander.send_hover_setpoint(0, 0, 0, target_height)
                                # print("Should have hovered to appropriate height now")
                            
                                #############
                                # Center ourselves first
                                #############

                                # target_y = 0                                            # Target y-coordinate
                                # y_moves = np.linspace(target_y, current_y, num = 7)     # You want to set it to go from 0 to the opposite of the current y position
                                # for y_pos in y_moves:                                   # Step
                                #     print("moving to y_pos: ", -y_pos)                  # Move and correct now - movement could be relative
                                #     cf.commander.send_position_setpoint(current_x, -y_pos, target_height, 0) # Syntax def send_position_setpoint(self, x, y, z, yaw) - https://www.bitcraze.io/documentation/repository/crazyflie-lib-python/master/api/cflib/crazyflie/commander/
                                #     time.sleep(0.5)
                                # print("Should be centered")

                                cf.commander.send_position_setpoint(current_x, 0, target_height, 0)

                                #############
                                # Hover
                                #############
                                for _ in range(10):
                                    cf.commander.send_hover_setpoint(0, 0, 0, target_height)
                                    time.sleep(0.1)
                                print("We have moved to the center of the system and hovered.")
                                # reached_height = True
                                centered = True
                                current_y = 0
                                print((current_x, current_y))
                            else:
                                #############
                                #Boolean for error
                                #############
                                # if horizontal_achieved:
                                #     _, _, box_width, box_height = checkforbook(frame, cf, cap)
                                #     current_x, current_y, forward_achieved = forward_controller(cf, box_width, box_height, current_x, current_y)
                                #     if forward_achieved:
                                #         break
                                # else:
                                #     center_x_of_book, _, _, _ = checkforbook(frame, cf, cap)
                                #     current_x, current_y, horizontal_achieved = horizontal_controller(cf, center_x_of_book, current_x, current_y)
                                if not first_frame_for_book:
                                    ret, frame = cap.read()
                                    if not ret:
                                        print("Error: Failed to capture frame.")
                                    first_frame_for_book = True
                                center_x_of_book, _, _, _ = checkforbook(frame, cf, cap)
                                current_x, current_y, horizontal_achieved = horizontal_controller(cf, center_x_of_book, current_x, current_y)
                                if horizontal_achieved:
                                    break
                            
                    
            if(elapsed > 200.0):
                break

        cap.release()

        # Descend and stop all motion:
        print(f"Okie good night (drop a fifty bag)")
        # time.sleep(5)
        # hover_and_descend(cf)
        # cf.commander.send_hover_setpoint(0, 0, 0, 0)

print('Done!')

Scanning interfaces for Crazyflies...
Crazyflies found:
radio://0/1/2M
radio://0/1/2M
radio://0/1/2M
radio://0/1/2M
radio://0/1/2M
radio://0/1/2M
radio://0/1/2M
radio://0/1/2M
radio://0/1/2M


[ WARN:0@2783.168] global cap_gstreamer.cpp:1173 isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


Initializing PID Controller
PID is done
Total area is 52172.5
Total area is 52882.0
Center check completed and yielded 52882.0
Sweeping right
Should have swept right
Total area is 53844.5
Total area is 54443.0
Right check completed and yielded 54443.0
Sweeping left
Should have swept left
Total area is 53093.5
Total area is 53653.0
Left check completed and yielded 53653.0
################################################################
Center position is best: 52882.0
Now we should have stayed at center
Current largest area is 16774.5
Current position is: x (forward/back): 0.0, y (left/right): 0
Current largest area is 24879.0
Current position is: x (forward/back): 0.25, y (left/right): 0
AIGHT WE GOTTA MOVE OUTTA THE WAY
Flag is down, Move right to avoid obstacle
Adjusting right to avoid obstacle
Current largest area is 29151.5
Current position is: x (forward/back): 0.25, y (left/right): -0.06666666666666667
AIGHT WE GOTTA MOVE OUTTA THE WAY
Flag is down, Move right to avoid obstacle
A

KeyboardInterrupt: 

In [ ]:
cap.release()

# Next Steps - As of 11/24

Best battery runs are above 4.19 volts.

~~Currently the drone moves right three steps upon detecting a reasonably sized red barrier. However, it lacks a few features that will be needed:~~
~~1  It does not move left like Jaylen Brown~~
~~2 It does not factor in the size of the boundary of the net -- it should be such that if it moves too far right given obstacles it should attempt moving left. It doesn't currently know its position to even factor in the size of the net or boundary, needed for step 2~~
~~3. Add a detection for left side net in case~~
~~5. Add a defensive functionality such that if the drone gets too close to the net at the far end (near where the book is) it moves backwards~~
~~1. Test feedback loop prevention - put a red barrier near the right end of the net, and see that if the drone is triggered to move left but detects a boundary, it shouldn't keep moving right it should keep moving left (GLOBAL FLAG VARIABLE THAT ESHAAN MENTIONED OVER TEXT ON 11/24)~~

0. Test the new code on a variety of setups. curious to see how it performs if you have a staggerred block of two pillars, one is forward and one is back. hte forward one is farther form net, the back one is closer to net. if these are both on the right side, and if the global flag is triggered for the drone to start moving left, might it crash in the pillar that is closer to the viewer? i am not sure
~~1. It lacks any functionality to detect a book and then land - look into optical flow or color-based search for this? Prefer color method (choose a blue, yellow or basically any color book that is not green, red, or white (or maybe black))~~
~~2. Add dynamical detection and speed... say the drone observes an object with bounding box of 15000 and an object with bounding box of 20000... in this case, it hsould proceed slowly around the 20000 but it can speed up until it gets close enough to 15000!~~
3. Continue tuning size of gap allowable
~~4. tune the parameter that tells drone to move backwards~~
5. One change that we could make is that if the center of the largest bounding box is on the left side of the drone's image then we move right and then otherwise if the center of the largest bounding box is on the right side of hte drone's image then we move left